# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
!pip install --upgrade azureml-sdk[automl]

Requirement already up-to-date: azureml-sdk[automl] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.24.0)


In [2]:
!pip install --upgrade --upgrade-strategy eager azureml-sdk

Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.24.0)
Requirement already up-to-date: azureml-train-automl-client~=1.24.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.24.0)
Requirement already up-to-date: azureml-dataset-runtime[fuse]~=1.24.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.24.0)
Requirement already up-to-date: azureml-core~=1.24.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.24.0.post2)
Requirement already up-to-date: azureml-train~=1.24.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.24.0)
Requirement already up-to-date: azureml-pipeline~=1.24.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.24.0)
Requirement already up-to-date: azureml-telemetry~=1.24.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-train-automl-c

Requirement already up-to-date: websocket-client>=0.32.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from docker->azureml-core~=1.24.0->azureml-sdk) (0.58.0)
Requirement already up-to-date: pyasn1>=0.1.1 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from ndg-httpsclient->azureml-core~=1.24.0->azureml-sdk) (0.4.8)
Requirement already up-to-date: ruamel.yaml.clib>=0.1.2; platform_python_implementation == "CPython" and python_version < "3.10" in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from ruamel.yaml>=0.15.35->azureml-core~=1.24.0->azureml-sdk) (0.2.2)
Requirement already up-to-date: cffi>=1.12 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from cryptography!=1.9,!=2.0.*,!=2.1.*,!=2.2.*,<4.0.0->azureml-core~=1.24.0->azureml-sdk) (1.14.5)
Requirement already up-to-date: certifi>=2017.4.17 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from requests<3.0.0,>=2.19.1->azureml-core~=1.24.0->azureml-sdk) (2020.12.5)
Requir

In [3]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

import pandas as pd
from azureml.widgets import RunDetails

In [4]:
azureml.core.__version__

'1.24.0'

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_experiment'

experiment=Experiment(ws, experiment_name)

In [6]:
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
dataset = pd.read_csv("heart_failure_clinical_records_dataset.csv")

dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [8]:
train, test = train_test_split(dataset, random_state=500, shuffle=True)

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)

Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_c7aa87162ca549d182c676e69d7f9c1b

In [9]:
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {    
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    training_data= train_data,
    featurization="auto",
    enable_early_stopping=True,
    label_column_name= "DEATH_EVENT",
    n_cross_validations=4,
    compute_target=cpu_cluster,
    blocked_models = ['XGBoostClassifier'],
    **automl_settings)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_8af90426-71d5-4962-9d1f-17b56aa9021f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_8af90426-71d5-4962-9d1f-17b56aa9021f',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-13T13:55:59.071681Z',
 'endTimeUtc': '2021-03-13T14:12:56.216291Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_experiment","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-140332","workspace_name":"quick-starts-ws-140332","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":null,"y_ma

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_automl_run, fitted_automl_model = remote_run.get_output()

best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_8af90426-71d5-4962-9d1f-17b56aa9021f_38,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
fitted_automl_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                        decision_function_shape='ovr',
                                                                                        degree=3,
                                                                                        gamma='scale',
                                                                        

In [15]:
#TODO: Save the best model
import joblib

joblib.dump(fitted_automl_model, 'outputs/automl_model.joblib')

['outputs/automl_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [35]:
from azureml.core.model import Model

# model = Model.register(workspace = ws,
#                         model_path ="outputs/automl_model.joblib",
#                         model_name = "automl_model")

model = remote_run.register_model(model_name = "automl_model")

In [36]:
from azureml.core.webservice import Webservice, AciWebservice

deploy_config = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=2,enable_app_insights=True)

In [37]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='./score.py')

In [39]:
service_name = 'automl-web-service'

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=deploy_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-13 14:59:35+00:00 Creating Container Registry if not exists.
2021-03-13 14:59:36+00:00 Use the existing image.
2021-03-13 14:59:36+00:00 Generating deployment configuration.
2021-03-13 14:59:38+00:00 Submitting deployment to compute..
2021-03-13 14:59:42+00:00 Checking the status of deployment automl-web-service..
2021-03-13 15:02:15+00:00 Checking the status of inference endpoint automl-web-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [42]:
import json
import requests

scoring_uri = service.scoring_uri
headers = {'Content-Type':'application/json'}

test_data_1 = json.dumps({'data':[{
    'age':75,
    'anaemia':0,
    'creatinine_phosphokinase':582,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':265000,
    'serum_creatinine':1.9,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

test_data_2 = json.dumps({'data':[{
    'age':40,
    'anaemia':0,
    'creatinine_phosphokinase':321,
    'diabetes':0,
    'ejection_fraction':35,
    'high_blood_pressure':0,
    'platelets':265000,
    'serum_creatinine':1,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':198}
    ]
        })


response = requests.post(scoring_uri, data=test_data_1, headers=headers)

print("Result 1:",response.text)


response = requests.post(scoring_uri, data=test_data_2, headers=headers)

print("Result 2:",response.text)

Result 1: [1]
Result 2: [0]


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs()

In [ ]:
service.delete()